In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from matplotlib.image import imread
import seaborn as sns
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Verileri Google Drive'dan çekme
from google.colab import drive
drive.mount('/content/drive')

# Veri yolu
data_path = '/content/drive/MyDrive/Datasets/NEU_metal_surface_defects/'

# Eğitim, doğrulama ve test veri yolları
train_path = os.path.join(data_path, 'train')
val_path = os.path.join(data_path, 'valid')
test_path = os.path.join(data_path, 'test')

# Sınıf isimleri
class_names = os.listdir(train_path)
num_classes = len(class_names)

# Veri yükleyici ve veri ön işleme
image_size = (200, 200)
batch_size = 32

train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    rescale=1./255,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

val_generator = test_datagen.flow_from_directory(
    val_path,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_path,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

# Model oluşturma
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(200, 200, 3))

model = Sequential()
model.add(base_model)
model.add(tf.keras.layers.GlobalAveragePooling2D())
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

# Eğitim
epochs = 50

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.n // train_generator.batch_size,
    epochs=epochs,
    validation_data=val_generator,
    validation_steps=val_generator.n // val_generator.batch_size
)

# Değerlendirme
test_loss, test_accuracy = model.evaluate(test_generator)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

# Sonuçları görselleştirme
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

Mounted at /content/drive
Found 1656 images belonging to 6 classes.
Found 72 images belonging to 6 classes.
Found 72 images belonging to 6 classes.
87910968/87910968 [==============================] - 4s 0us/step
Epoch 1/50
51/51 [==============================] - 1123s 21s/step - loss: 0.5107 - accuracy: 0.8547 - val_loss: 10.0101 - val_accuracy: 0.2031
Epoch 2/50
51/51 [==============================] - 763s 15s/step - loss: 0.2218 - accuracy: 0.9440 - val_loss: 24.2560 - val_accuracy: 0.6562
Epoch 3/50
51/51 [==============================] - 746s 15s/step - loss: 0.1882 - accuracy: 0.9470 - val_loss: 21.1058 - val_accuracy: 0.7812
Epoch 4/50
 4/51 [=>............................] - ETA: 11:24 - loss: 0.1789 - accuracy: 0.9453

KeyboardInterrupt: ignored